# Matsch Valley
## NDVI - Precipitation - Land Surface Temperature

This is an example on how to use [Google Earth Engine](https://earthengine.google.com) via [Python](https://gee-python-api.readthedocs.io/en/latest/index.html) to visualize and download different kinds of data for specific locations.


Got inspired [here](https://developers.google.com/earth-engine/tutorials/community/time-series-visualization-with-altair)

**Contact**: Giulio Genova ([Twitter](https://twitter.com/giuliogenova), [GitHub](https://github.com/GiulioGenova), [Linkedin](https://www.linkedin.com/in/giulio-genova-a54b30a4/))

In [ ]:
import ee
import geemap
from geemap import ee_initialize
import os
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

ee_initialize()

In [ ]:

# User defined functions
from create_reduce_region_function import create_reduce_region_function
from fc_to_dict import fc_to_dict
from add_date_info import add_date_info
from addNDVI import addNDVI
from do_buffer import do_buffer

In [ ]:
stations = ee.FeatureCollection([
  ee.Feature(ee.Geometry.Point(10.5798451965, 46.6862577024), {'name': 'B2 (1490m)'}),
  ee.Feature(ee.Geometry.Point(10.5918853513, 46.6917328627), {'name': 'B3 (1930m)'}),
  ee.Feature(ee.Geometry.Point(10.5850561988, 46.684249288), {'name': 'P2 (1549m)'})])

aoi = stations.map(do_buffer)
aoi_all = aoi.geometry()
aoi_sel = ee.Geometry.Point(10.5850561988, 46.684249288)

In [ ]:
s2 = ee.ImageCollection('COPERNICUS/S2') \
    .filterDate('2016-01-01', '2020-11-20') \
    .filterBounds(aoi_all) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 25))

In [ ]:
ndvi = s2.map(addNDVI).select("NDVI")
ndvi_bands = ndvi.toBands().clip(aoi_all)

In [ ]:
ndvi_vis = {
  'min': -1.0,'max': 1,
  'palette': ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', 
  '74A901','66A000', '529400', '3E8601', '207401', '056201', '004C00', 
  '023B01','012E01', '011D01', '011301']
}

Map = geemap.Map(center=[ 46.688,10.586], zoom = 15)
Map.add_basemap("SATELLITE")
Map.addLayer(ndvi_bands,name="NDVI_series")
Map.addLayer(ndvi_bands.select(18),ndvi_vis,name="NDVI_image")
Map.addLayer(stations,name="Monitoring Stations")
Map.set_plot_options(plot_type='bar',add_marker_cluster=False, marker=None)
Map.roi_reducer = ee.Reducer.mean()
Map

## NDVI for station P2

Here we download and prepare 5 years of NDVI data from [Sentinel 2](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2?hl=en) for a specific location (median in the window around the point)

In [ ]:
reduce_ndvi = create_reduce_region_function(
    geometry=aoi_sel, reducer=ee.Reducer.median(), scale=10)

ndvi_stat_fc = ee.FeatureCollection(ndvi.map(reduce_ndvi)).filter(
    ee.Filter.notNull(ndvi.first().bandNames()))

ndvi_dict = fc_to_dict(ndvi_stat_fc).getInfo()
ndvi_df = pd.DataFrame(ndvi_dict)

ndvi_df = add_date_info(ndvi_df).drop(['millis','system:index'],axis=1)
ndvi_df = ndvi_df.drop_duplicates(["NDVI","Year","DOY"])
ndvi_df.head(5)

In [ ]:
fig = px.line(ndvi_df, x="DOY", y="NDVI", color='Year')
f = go.FigureWidget(fig)
f

## Precipitation for station P2

Here we download and prepare 5 years of Precipitation data from [NASA](https://developers.google.com/earth-engine/datasets/catalog/NASA_GPM_L3_IMERG_MONTHLY_V06) for a specific location (pixel size is actually bigger than the window around the point)


In [ ]:
prec = ee.ImageCollection('NASA/GPM_L3/IMERG_MONTHLY_V06') \
                  .filter(ee.Filter.date('2016-01-01', '2020-11-20'))

reduce_prec = create_reduce_region_function(
    geometry=aoi_sel, reducer=ee.Reducer.median(), scale=5000)

prec_stat_fc = ee.FeatureCollection(prec.map(reduce_prec)).filter(
    ee.Filter.notNull(prec.first().bandNames()))

prec_dict = fc_to_dict(prec_stat_fc).getInfo()
prec_df = pd.DataFrame(prec_dict)

prec_df = add_date_info(prec_df).drop(['millis','system:index'],axis=1)
prec_df['precipitation'] = prec_df['precipitation']*1000
prec_df.head(5)



In [ ]:
fig_prec = px.bar(prec_df, x="Timestamp", y="precipitation", color='Year')
f_prec = go.FigureWidget(fig_prec)
f_prec

## Land Surface Temperature (Day) for station P2

Here we download and prepare 5 years of Land Surface Temperature data from [MODIS](https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MOD11A2) for a specific location (pixel size is actually bigger than the window around the point)

In [ ]:
lst = ee.ImageCollection('MODIS/006/MOD11A2') \
                  .filter(ee.Filter.date('2016-01-01', '2020-11-20'))

reduce_lst = create_reduce_region_function(
    geometry=aoi_sel, reducer=ee.Reducer.median(), scale=5000)

lst_stat_fc = ee.FeatureCollection(lst.map(reduce_lst)).filter(
    ee.Filter.notNull(lst.first().bandNames()))

lst_dict = fc_to_dict(lst_stat_fc).getInfo()
lst_df = pd.DataFrame(lst_dict)

lst_df = add_date_info(lst_df).drop(['millis','system:index'],axis=1)
lst_df['LST_Day_1km'] = lst_df['LST_Day_1km']/1000
lst_df['LST_Night_1km'] = lst_df['LST_Night_1km']/1000
lst_df.head(5)

In [ ]:
fig_lst = px.line(lst_df, x="Timestamp", y="LST_Day_1km", color='Year')
f_lst = go.FigureWidget(fig_lst)
f_lst